# Example Notebook: Minimal Working Title

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from scipy.stats import chi2
import sys
import os

# Get the parent directory of the current working directory in BINDER
SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
CODE_DIR = os.path.join(SOURCE_DIR, "src")
RES_DIR = os.path.join(SOURCE_DIR, "res")
sys.path.append(CODE_DIR)
sys.path.append(RES_DIR)

# Import pyTENAX
from pyTENAX import smev, tenax, plotting

In [ ]:
S = tenax.TENAX(
    return_period=[
        2,
        5,
        10,
        20,
        50,
        100,
        200,
    ],  # for some reason it doesnt like calculating RP =<1
    durations=[10, 60, 180, 360, 720, 1440],
    time_resolution=5,  # time resolution in minutes
    left_censoring=[0, 0.90],
    alpha=0.05,
)

In [ ]:
file_path_input = "res/prec_data_Aadorf.parquet"
# Load data from csv file
data = pd.read_parquet(os.path.join(RES_DIR, "prec_data_Aadorf.parquet"))

In [ ]:
# Convert 'prec_time' column to datetime, if it's not already
data["prec_time"] = pd.to_datetime(data["prec_time"])
# Set 'prec_time' as the index
data.set_index("prec_time", inplace=True)
name_col = "prec_values"  # name of column containing data to extract

In [ ]:
# push values belows 0.1 to 0 in prec due to
data.loc[data[name_col] < S.min_rain, name_col] = 0

In [ ]:
data = S.remove_incomplete_years(data, name_col)

In [ ]:
# get data from pandas to numpy array
df_arr = np.array(data[name_col])
df_dates = np.array(data.index)

In [ ]:
# extract indexes of ordinary events
# these are time-wise indexes =>returns list of np arrays with np.timeindex
idx_ordinary = S.get_ordinary_events(
                data=df_arr, dates=df_dates, name_col=name_col, check_gaps=False
                )

In [ ]:
# get ordinary events by removing too short events
# returns boolean array, dates of OE in TO, FROM format, and count of OE in each years
arr_vals, arr_dates, n_ordinary_per_year = S.remove_short(idx_ordinary)

In [ ]:
# assign ordinary events values by given durations, values are in depth per duration, NOT in intensity mm/h
dict_ordinary, dict_AMS = S.get_ordinary_events_values(
                            data=df_arr, dates=df_dates, arr_dates_oe=arr_dates
                            )

In [ ]:
# load temperature data
t_data = pd.read_parquet(os.path.join(RES_DIR, "temp_data_Aadorf.parquet"))
# Convert 'temp_time' column to datetime if it's not already in datetime format
t_data["temp_time"] = pd.to_datetime(t_data["temp_time"])
# Set 'temp_time' as the index
t_data.set_index("temp_time", inplace=True)
temp_name_col = "temp_values"

In [ ]:
df_arr_t_data = np.array(t_data[temp_name_col])
df_dates_t_data = np.array(t_data.index)

In [ ]:
dict_ordinary, _, n_ordinary_per_year = S.associate_vars(
    dict_ordinary, df_arr_t_data, df_dates_t_data
)

In [ ]:
# Your data (P, T arrays) and threshold thr=3.8
P = dict_ordinary["10"]["ordinary"].to_numpy()  # Replace with your actual data
T = dict_ordinary["10"]["T"].to_numpy()  # Replace with your actual data
blocks_id = dict_ordinary["10"]["year"].to_numpy()  # Replace with your actual data
# Number of threshold
thr = dict_ordinary["10"]["ordinary"].quantile(S.left_censoring[1])

In [ ]:
# Sampling intervals for the Montecarlo
Ts = np.arange(
    np.min(T) - S.temp_delta, np.max(T) + S.temp_delta, S.temp_res_monte_carlo
)

In [ ]:
# TENAX MODEL HERE
# magnitude model
F_phat, loglik, _, _ = S.magnitude_model(P, T, thr)
# temperature model
g_phat = S.temperature_model(T)
# M is mean n of ordinary events
n = n_ordinary_per_year.sum() / len(n_ordinary_per_year)
# estimates return levels using MC samples
RL, _, P_check = S.model_inversion(F_phat, g_phat, n, Ts)

In [ ]:
# PLOTTING THE GRAPHS
eT = np.arange(
    np.min(T), np.max(T) + 4, 1
)  # define T values to calculate distributions. +4 to go beyond graph end

# fig 2a
qs = [0.85, 0.95, 0.99, 0.999]
plotting.TNX_FIG_magn_model(P, T, F_phat, thr, eT, qs)
plt.ylabel("10-minute precipitation (mm)")
plt.title("fig 2a")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2))
plt.show()

# fig 2b
hist, pdf_values = tenax.TNX_FIG_temp_model(T, g_phat, S.beta, eT)
plt.title("fig 2b")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2))
plt.show()